<a href="https://colab.research.google.com/github/AndreaBertoglio/MLDM/blob/master/NeuralNetwork_%26FeatureConstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory




In [ ]:
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, normalize, StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, r2_score
from sklearn.svm import SVC

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, RMSprop
from keras.losses import sparse_categorical_crossentropy, binary_crossentropy
from keras.metrics import Accuracy

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/serivan/mldmlab/master/Datasets/Kaggle2020/train.csv')
data

,Id,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,Quality
0,3940,6.4,0.39,0.21,1.2,41.00,35.0,136.0,0.99225,3.15,0.46,10.2,Disappointing
1,1655,7.5,305.00,0.40,18.9,59.00,44.0,170.0,1.00000,2.99,0.46,9.0,Disappointing
2,1867,6.3,0.28,0.30,3.1,39.00,24.0,115.0,0.99420,3.05,0.43,8.6,Disappointing
3,4476,7.4,0.18,0.30,10.4,45.00,44.0,174.0,0.99660,3.11,0.57,9.7,Good
4,453,6.9,0.20,0.36,1.5,31.00,38.0,147.0,0.99310,3.35,0.56,11.0,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3484,2768,6.9,0.14,0.38,1.0,41.00,22.0,81.0,0.99043,3.03,0.54,11.4,Good
3485,4347,9.0,0.20,0.33,3.5,49.00,10.0,40.0,0.99440,3.14,0.36,9.8,Good
3486,1870,7.6,0.29,0.26,6.5,42.00,32.0,160.0,0.99440,3.14,0.47,10.7,Disappointing
3487,613,7.5,0.17,0.32,1.7,0.04,51.0,148.0,0.99160,3.21,0.44,11.5,Good


In [ ]:
data["Quality"] = np.where(data["Quality"].str.contains("Good"), 1, 0)

train_y = data["Quality"]

citricacid = data['fixed.acidity'] * data['citric.acid']
citric_acidity = pd.DataFrame(citricacid, columns=['citric_accidity'])

density_acidity = data['fixed.acidity'] * data['density']
density_acidity = pd.DataFrame(density_acidity, columns=['density_acidity'])


datafinal = data.join(citric_acidity).join(density_acidity)
predictor_cols = ['fixed.acidity','volatile.acidity','citric.acid','residual.sugar','chlorides','free.sulfur.dioxide','total.sulfur.dioxide','density','pH','sulphates','alcohol','density_acidity','citric_accidity']

# La x sono gli attributi
train_X = datafinal[predictor_cols]

# Sostituisce i missing values con la media e lo applica alle x

#imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = IterativeImputer(missing_values=np.nan, max_iter=30)
imp = imp.fit(train_X)


# Impute our data, then train
train_X_imp = imp.transform(train_X)

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(train_X_imp, train_y,  random_state = 0) #train_size = 0.8

In [ ]:
scale = StandardScaler()

scaledtrain = scale.fit_transform(xTrain)
scaledtest = scale.transform(xTest)

In [ ]:


NeuralModel = Sequential([
                          Dense(128, activation='relu', input_shape=(13,)),
                          Dense(32, activation='relu'),
                          Dense(64, activation='relu'),
                          Dense(64, activation='relu'),
                          Dense(64, activation='relu'),
                          Dense(1, activation='sigmoid')
])

rms = Adam(lr=0.0003)

model = NeuralModel.compile(optimizer=rms, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
hist = NeuralModel.fit(scaledtrain, yTrain, epochs=1000, validation_data=(scaledtest,yTest))

Train on 2616 samples, validate on 873 samples
Epoch 1/1000
2616/2616 [==============================] - 0s 65us/step - loss: 0.0046 - accuracy: 0.9989 - val_loss: 1.5841 - val_accuracy: 0.7904
Epoch 2/1000
2616/2616 [==============================] - 0s 65us/step - loss: 0.0032 - accuracy: 0.9989 - val_loss: 1.6204 - val_accuracy: 0.7927
Epoch 3/1000
2616/2616 [==============================] - 0s 62us/step - loss: 0.0047 - accuracy: 0.9981 - val_loss: 1.6109 - val_accuracy: 0.7881
Epoch 4/1000
2616/2616 [==============================] - 0s 56us/step - loss: 0.0052 - accuracy: 0.9989 - val_loss: 1.7141 - val_accuracy: 0.7938
Epoch 5/1000
2616/2616 [==============================] - 0s 56us/step - loss: 0.0139 - accuracy: 0.9962 - val_loss: 1.6448 - val_accuracy: 0.7778
Epoch 6/1000
2616/2616 [==============================] - 0s 58us/step - loss: 0.0145 - accuracy: 0.9954 - val_loss: 1.7227 - val_accuracy: 0.7835
Epoch 7/1000
2616/2616 [==============================] - 0s 55us/step 

In [ ]:
def generateSubmission(myModel, submissionFile, description):
    # Read the test data
    test = pd.read_csv('https://raw.githubusercontent.com/serivan/mldmlab/master/Datasets/Kaggle2020/train.csv')
    
    citricacid = test['fixed.acidity'] * test['citric.acid']
    citric_acidity = pd.DataFrame(citricacid, columns=['citric_accidity'])

    density_acidity = test['fixed.acidity'] * data['density']
    density_acidity = pd.DataFrame(density_acidity, columns=['density_acidity'])


    testF = test.join(citric_acidity).join(density_acidity)
    

    # Treat the test data in the same way as training data. In this case, pull same columns.
    test_X = testF[predictor_cols]

    # Impute each test item, then predict
    test_X_imp = imp.transform(test_X)
    
    # Use the model to make predictions
    predicted_q = myModel.predict(test_X_imp)
    # We will look at the predicted Qualities to ensure we have something sensible.
    print(predicted_q)
    
    #submission file
    my_submission = pd.DataFrame({'Id': test.Id, 'Quality': predicted_q})
    # you could use any filename. We choose submission here
    my_submission.to_csv(submissionFile, index=False)
    

In [ ]:
generateSubmission(NeuralModel, "NNproAP", "ciao")

[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [1.]]


Exception: ignored